# Global Forest Change


- **Special requirements:** A Google account, access to Google Earth Engine.

- **Prerequisites:** You should have completed the "Week 5 - Prac 1" and "Week 5 - Prac 2" notebooks.

## Description
The Hansen et al. (2013) Global Forest Change dataset represents forest change, at 30 meters resolution, globally, during the years (2000-Present). The GEE documentation is [here](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2022_v1_10). You can read more on the usage notes for this iteration of the dataset [here](https://storage.googleapis.com/earthenginepartners-hansen/GFC-2022-v1.10/download.html)

In this session, you will learn how to work with this data to analyse and quantify Forest Change.

The paper describing this dataset can be found [here](https://www.science.org/doi/full/10.1126/science.1244693?casa_token=HKtLohSM6y4AAAAA%3AIgaZz5d3pLw4oipkk-HdMLSUbwPoM1XJlpHS9CpXFP_-BoVO0vRtRmAXlP4z0ls49gnkGBmIWDFq)

## Aims of the practical session
* Create ROI and load it
* Load the Global Forest Change dataset and select the bands of interest
* Calculate and display both forest loss and gain
* Compute the loss/gain for each year

## Load packages

Import GEE packages that are needed for the analysis.

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
# ee.Authenticate()

## Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayerControl()

## Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map. In this example, we'll import a geojson from github that marks the boundary of NSW region.

In [ ]:
# # # load NSW shapefile
path = 'https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/NSW.geojson'
geometry = geemap.geojson_to_ee(path)
# Map.addLayer(geometry, {}, 'NSW Region')
Map.centerObject(geometry);

## Load Global Forest Change layer
Using the below code read the Global Forest Change layer and load it. Then, clip the layer based on ROI and visualize it on GEEmap.

> Note: If we simply plotted the Forest Change layer with its default parameters, areas without forest would show up as black (areas where data = 0). For example, there are no trees in the ocean. To make these areas transparent, you can mask the no-data values. Every pixel in Earth Engine has both a value and a mask. The image is rendered with transparency set by the mask, with zero being completely transparent and one being completely opaque.  i.e., You can mask an image with itself. If you mask the `treecover2000` band with itself, all the areas in which forest cover is zero will be transparent.

In [ ]:
# # # load new version of Global Forest Change Layer 
gfc = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')

# Display forest cover in the year 2000 with 'greens', depending on % canopy cover
Map.addLayer(gfc.mask(gfc).clip(geometry),
             {'bands': ['treecover2000'],
             'palette': ['000000', '00FF00'],
             'max':100
             },
             'treecover2000')
Map

## Display forest loss and forest gain
We'd like **forest loss to show up as bright red** and **forest gain to show up as bright blue**. The best way to do this is to select each image (treecover, loss, gain) seperately and add them to the map. This is better than making a 3-band RGB image as each colour will show up more prominently.

In [ ]:
#select each of the different bands
treeCover = gfc.select(['treecover2000']).clip(geometry)
lossImage = gfc.select(['loss']).clip(geometry)
gainImage = gfc.select(['gain']).clip(geometry)

## Add the tree cover layer in green.
Map.addLayer(treeCover.updateMask(treeCover),
    {'palette': ['000000', '00FF00'], 'max': 100}, 'Forest Cover');

## Add the loss layer in red.
Map.addLayer(lossImage.updateMask(lossImage),
            {'palette': ['FF0000']}, 'Loss');

## Add the gain layer in blue.
Map.addLayer(gainImage.updateMask(gainImage),
            {'palette': ['0000FF']}, 'Gain');

Map

<div class="alert alert-block alert-danger"> 

## Exercise 1

- For a country or region where you think forest loss and gain might be important, create a map of forest change (loss and gain).  You can use the [Large Scale International Boundary (LSIB)](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017) dataset to select the country of interest.  Use the `and()` method to create the lossAndGain image and display pixels where both loss and gain occur and then discuss the changes you see, with reference to the locations and patterns of forest change, and briefly hypothesize why these changes might be occurring (using referencing if you consult literature).  To help you understand the changes, consider loading true and/or false color composite satellite images over the study region from different times.
    
More information: The GFC dataset has a band whose pixels are 1 where loss occurred and 0 otherwise, and a band that is 1 where gain has occurred and a 0 otherwise. To create a band where pixels in both the loss and the gain bands have a 1, you can use the `and()` logical method on images. The `and()` method is called like image1.and(image2) and returns an image in which pixels are 1 where both image1 and image2 are 1, and 0 elsewhere.
    
    
</div>    

## Quantifying Forest Change in a Region of Interest
To help compute areas, Earth Engine has the `ee.Image.pixelArea()` method which generates an image in which the value of each pixel is the pixel's area in square meters. Multiplying the loss image with this area image and then summing over the result gives us a measure of area.

In [ ]:
# Get the forest loss image.
areaLossImage = lossImage.multiply(ee.Image.pixelArea())

In [ ]:
# Sum the values of forest loss pixels in the ROI.
stats = areaLossImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(), #add up all the piels == 1
  'geometry': geometry,
  'scale': 30,
  'maxPixels': 1e13
})

In [ ]:
classAreas = ee.List(stats.get('loss'))
print('Total forest loss in NSW from 2000-2022: ', round(classAreas.getInfo() / 1000000, 2), 'square kms')

## Calculating Annual Forest Loss

In the previous section you learned how to calculate total forest area lost over the entire study period (2000-2022) in the given region of interest (NSW) using the `reduceRegion` method. Instead of calculating the total loss, it would be helpful to compute the loss for each year. The way to achieve this in Earth Engine is using a `Grouped Reducer`.

We will:

* Using the `lossyear` band in the dataset, determine the area of forest loss in each year
* Data wrangle the result into a pandas dataframe so we can make a nice plot of forest loss per year (in square kilometres)

In [ ]:
lossYear = gfc.select(['lossyear'])

# Add the lossyear to the AreaLossImage and reduce over a 'group' (years)
lossByYear = areaLossImage.addBands(lossYear).reduceRegion(**{
  'reducer': ee.Reducer.sum().group(**{
    'groupField': 1
    }),
  'geometry': geometry,
  'scale': 30,
  'maxPixels': 1e13
})

print(lossByYear.getInfo())

### Convert/format data for plotting

In [ ]:
# format the output to make the result a dictionary, with year as the key and loss area as the value.
classAreas = ee.List(lossByYear.get('groups'))

#funtion for creating a dictionary where year is the key and area is the value
def func_dic(item):
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('group')).format("20%02d")
    area = ee.Number(areaDict.get('sum')).divide(1e6) #square kilometress
    return ee.List([classNumber, area])

#map the function over the result
classAreaLists = classAreas.map(func_dic)
result = ee.Dictionary(classAreaLists.flatten())

#convert into a pandas dataframe
class_areas = pd.DataFrame(classAreaLists.getInfo(), columns = ['Year','Area'])
class_areas.head()

### Plot annual forest loss

In [ ]:
class_areas.plot(x ='Year', y='Area', kind='bar', figsize=(11,5))
plt.ylabel('Forest Loss (km2)')

<div class="alert alert-block alert-danger"> 

## Exercise 2

- Select another Australian state as your ROI and then calculate annual forest loss for the whole period of data availability (2000-2022), plotting the results. Then, discuss the trends you see.
    
- Compare the results of forest loss in Australia with the country/region you selected in Exercise 1. Briefly discuss the similarities and/or differences in the rates and drivers of forest loss between the two regions.
    
</div>    

<!-- <div class="alert alert-block alert-danger"> 

## Challenge

- Create a multi-temporal color composite of treeCover to highlight areas that have undergone deforestation across two different years. You can pick up the first year of the time series and the last year (or two consecutive years).  To construct a temporal color composite, add the first image date to the R colour channel and the second to the G and B channels.  There is an example of creating a multi-temporal color composite in your lecture notes from this week.

    
</div>     -->

## References
This is where the references go. For exmaple:

* Hansen, M. C., Potapov, P. V., Moore, R., Hancher, M., Turubanova, S. A., Tyukavina, A., ... & Townshend, J. (2013). High-resolution global maps of 21st-century forest cover change. science, 342(6160), 850-853.

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site 

**Last modified:** September 2023